In [4]:
import os
import csv
import json
import tkinter as tk
from tkinter import messagebox, simpledialog
from datetime import datetime
from ttkbootstrap import Style
from ttkbootstrap.constants import *

# File paths
DATA_FILE = "students.json"
BACKUP_FILE = "students_backup.json"
CSV_FILE = "students.csv"
ADMIN_PASSWORD = "admin123"  # Change this to your own

# Load student data
def load_students():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            return json.load(f)
    return []

# Save student data
def save_students():
    with open(DATA_FILE, "w") as f:
        json.dump(student_list, f, indent=2)

# Backup
def backup_data():
    with open(BACKUP_FILE, "w") as f:
        json.dump(student_list, f, indent=2)
    messagebox.showinfo("Backup", "Backup created successfully.")

# Restore
def restore_data():
    if os.path.exists(BACKUP_FILE):
        with open(BACKUP_FILE, "r") as f:
            restored = json.load(f)
        student_list.clear()
        student_list.extend(restored)
        save_students()
        update_display()
        messagebox.showinfo("Restore", "Backup restored successfully.")
    else:
        messagebox.showwarning("Restore", "No backup found.")

# Add student
def add_student():
    name = name_entry.get().strip()
    if name:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        student_list.append({"name": name, "date": timestamp})
        save_students()
        name_entry.delete(0, tk.END)
        update_display()

# Remove student
def remove_student():
    selected = student_listbox.curselection()
    if selected:
        del student_list[selected[0]]
        save_students()
        update_display()

# Edit student
def edit_student():
    selected = student_listbox.curselection()
    if selected:
        index = selected[0]
        original = student_list[index]["name"]
        new_name = simpledialog.askstring("Edit", "Edit student name:", initialvalue=original)
        if new_name:
            student_list[index]["name"] = new_name
            student_list[index]["date"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            save_students()
            update_display()

# Sort A-Z
def sort_az():
    student_list.sort(key=lambda x: x["name"].lower())
    update_display()

# Sort Z-A
def sort_za():
    student_list.sort(key=lambda x: x["name"].lower(), reverse=True)
    update_display()

# Export to CSV
def export_csv():
    with open(CSV_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Date Added"])
        for student in student_list:
            writer.writerow([student["name"], student["date"]])
    messagebox.showinfo("Export", "Exported to students.csv")

# Search functionality
def search_student(event=None):
    keyword = search_entry.get().strip().lower()
    if keyword:
        filtered = [s for s in student_list if keyword in s["name"].lower()]
        update_display(filtered)
    else:
        update_display()

# Update GUI display
def update_display(data=None):
    student_listbox.delete(0, tk.END)
    show_data = data if data else student_list
    for student in show_data:
        display = f"{student['name']} — {student['date']}"
        student_listbox.insert(tk.END, display)
    count_label.config(text=f"Total Students: {len(show_data)}")

# Admin login check
def login_prompt():
    pwd = simpledialog.askstring("Admin Login", "Enter admin password:", show='*')
    if pwd == ADMIN_PASSWORD:
        return True
    else:
        messagebox.showerror("Access Denied", "Incorrect password.")
        root.destroy()
        return False

# --- GUI Setup ---
root = tk.Tk()
root.title("🧑‍🎓 Student Registration System")
root.geometry("620x700")

# Bootstrap Dark Style
style = Style(theme="darkly")
style.master.configure(bg=style.colors.bg)

# Login Check
if not login_prompt():
    exit()

# Load data
student_list = load_students()

# Title
tk.Label(root, text="📋 Student Registration System", font=("Helvetica", 18, "bold"), bg=style.colors.bg, fg="white").pack(pady=20)

# Entry
name_entry = tk.Entry(root, font=("Arial", 14), width=30)
name_entry.pack(pady=5)

# Buttons: Add, Edit, Remove
frame1 = tk.Frame(root, bg=style.colors.bg)
frame1.pack(pady=5)
tk.Button(frame1, text="➕ Add", command=add_student, width=15).grid(row=0, column=0, padx=5)
tk.Button(frame1, text="✏ Edit", command=edit_student, width=15).grid(row=0, column=1, padx=5)
tk.Button(frame1, text="❌ Remove", command=remove_student, width=15).grid(row=0, column=2, padx=5)

# Sort Buttons
frame2 = tk.Frame(root, bg=style.colors.bg)
frame2.pack(pady=5)
tk.Button(frame2, text="🔼 Sort A-Z", command=sort_az, width=15).grid(row=0, column=0, padx=5)
tk.Button(frame2, text="🔽 Sort Z-A", command=sort_za, width=15).grid(row=0, column=1, padx=5)

# Search
tk.Label(root, text="🔍 Search", font=("Arial", 12), bg=style.colors.bg, fg="white").pack()
search_entry = tk.Entry(root, font=("Arial", 12), width=30)
search_entry.pack()
search_entry.bind("<KeyRelease>", search_student)

# Listbox
tk.Label(root, text="🎓 Registered Students", font=("Arial", 12, "bold"), bg=style.colors.bg, fg="white").pack(pady=10)
student_listbox = tk.Listbox(root, height=12, width=60, font=("Arial", 12),
                             bg="#2e2e2e", fg="white", selectbackground="gray")
student_listbox.pack(pady=5)

# Count Label
count_label = tk.Label(root, text="", font=("Arial", 11), bg=style.colors.bg, fg="white")
count_label.pack(pady=5)

# Export / Backup / Restore
frame3 = tk.Frame(root, bg=style.colors.bg)
frame3.pack(pady=10)
tk.Button(frame3, text="📤 Export CSV", command=export_csv, width=15).grid(row=0, column=0, padx=5)
tk.Button(frame3, text="💾 Backup", command=backup_data, width=15).grid(row=0, column=1, padx=5)
tk.Button(frame3, text="♻ Restore", command=restore_data, width=15).grid(row=0, column=2, padx=5)

# Footer
tk.Label(root, text="Made with ❤ using Python + Tkinter + ttkbootstrap", font=("Arial", 9), bg=style.colors.bg, fg="#aaaaaa").pack(pady=20)

# Initial load
update_display()
root.mainloop()

TclError: invalid command name "."